In [ ]:
# Problem statement
# 14 controls
# 1 target
# 5 auxilliary 

# b0-13, t, a0-4

# Design 1: (3-4-3) 
# 4 3-bit in place control gates (b0-11)->(a0-3)
# 1 4-bit in place control gate taking 2 remaining bits and 2 auxiliary bits (b12-13, a0-1) -> a4
# 3 3-bit in place control gate that takes (a2-4) -> t


In [ ]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram 

In [48]:
control_qubits = QuantumRegister(14, name="c")
target_qubit = QuantumRegister(1, name='t')
ancilla_qubits = QuantumRegister(5, name='a')
classical_bits = ClassicalRegister(1, name='out')
qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)

#qc.mcx()

In [42]:
# no optimization - design 0
qc.mcx(control_qubits, target_qubit[0])
qc.measure(target_qubit[0], classical_bits[0])

In [43]:
qc.draw()

c_0: ──■─────
         │     
  c_1: ──■─────
         │     
  c_2: ──■─────
         │     
  c_3: ──■─────
         │     
  c_4: ──■─────
         │     
  c_5: ──■─────
         │     
  c_6: ──■─────
         │     
  c_7: ──■─────
         │     
  c_8: ──■─────
         │     
  c_9: ──■─────
         │     
 c_10: ──■─────
         │     
 c_11: ──■─────
         │     
 c_12: ──■─────
         │     
 c_13: ──■─────
       ┌─┴─┐┌─┐
    t: ┤ X ├┤M├
       └───┘└╥┘
  a_0: ──────╫─
             ║ 
  a_1: ──────╫─
             ║ 
  a_2: ──────╫─
             ║ 
  a_3: ──────╫─
             ║ 
  a_4: ──────╫─
             ║ 
out: 1/══════╩═
             0

In [44]:
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'])
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

81916
OrderedDict([('u', 49150), ('cx', 49148), ('measure', 1)])


In [ ]:
# design 1 

for i in range(14):
    qc.x(control_qubits[i])

for i in range(4):
    qc.mcx(control_qubits[3*i:3*(i+1)], ancilla_qubits[i])

qc.mcx(control_qubits[12:14] + ancilla_qubits[0:2], ancilla_qubits[4])
qc.mcx(ancilla_qubits[2:5], target_qubit[0])
qc.measure(target_qubit[0], classical_bits[0])


In [49]:
# design 2 
for i in range(14):
    qc.x(control_qubits[i])

for i in range(3):
    qc.mcx(control_qubits[3*i:3*(i+1)], ancilla_qubits[i])
# 0 - 8
qc.mcx(control_qubits[9:13], ancilla_qubits[3])

qc.mcx(ancilla_qubits[0:2] + control_qubits[13:14], ancilla_qubits[4])

qc.mcx(ancilla_qubits[2:5], target_qubit[0])
qc.measure(target_qubit[0], classical_bits[0])

In [50]:
qc.draw()

┌───┐                                 
  c_0: ┤ X ├──■──────────────────────────────
       ├───┤  │                              
  c_1: ┤ X ├──■──────────────────────────────
       ├───┤  │                              
  c_2: ┤ X ├──■──────────────────────────────
       ├───┤  │                              
  c_3: ┤ X ├──┼────■─────────────────────────
       ├───┤  │    │                         
  c_4: ┤ X ├──┼────■─────────────────────────
       ├───┤  │    │                         
  c_5: ┤ X ├──┼────■─────────────────────────
       ├───┤  │    │                         
  c_6: ┤ X ├──┼────┼────■────────────────────
       ├───┤  │    │    │                    
  c_7: ┤ X ├──┼────┼────■────────────────────
       ├───┤  │    │    │                    
  c_8: ┤ X ├──┼────┼────■────────────────────
       ├───┤  │    │    │                    
  c_9: ┤ X ├──┼────┼────┼────■───────────────
       ├───┤  │    │    │    │               
 c_10: ┤ X ├──┼────┼────┼────■───────────────
       ├───┤  │    │    │    │               
 c_11: ┤ X ├──┼────┼────┼────■───────────────
       ├───┤  │    │    │    │               
 c_12: ┤ X ├──┼────┼────┼────■───────────────
       ├───┤  │    │    │    │               
 c_13: ┤ X ├──┼────┼────┼────┼────■──────────
       └───┘  │    │    │    │    │  ┌───┐┌─┐
    t: ───────┼────┼────┼────┼────┼──┤ X ├┤M├
            ┌─┴─┐  │    │    │    │  └─┬─┘└╥┘
  a_0: ─────┤ X ├──┼────┼────┼────■────┼───╫─
            └───┘┌─┴─┐  │    │    │    │   ║ 
  a_1: ──────────┤ X ├──┼────┼────■────┼───╫─
                 └───┘┌─┴─┐  │    │    │   ║ 
  a_2: ───────────────┤ X ├──┼────┼────■───╫─
                      └───┘┌─┴─┐  │    │   ║ 
  a_3: ────────────────────┤ X ├──┼────■───╫─
                           └───┘┌─┴─┐  │   ║ 
  a_4: ─────────────────────────┤ X ├──■───╫─
                                └───┘      ║ 
out: 1/════════════════════════════════════╩═
                                           0

In [51]:
aer_simulator = Aer.get_backend('aer_simulator')
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'])
qobj = assemble(qc_transpiled)
result = aer_simulator.run(qobj).result()
print(result.get_counts())

{'1': 1024}


In [ ]:
qc_transpiled.draw()

In [ ]:
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

115
OrderedDict([('u', 119), ('cx', 106), ('measure', 1)])


In [53]:
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

92
OrderedDict([('u', 119), ('cx', 106), ('measure', 1)])
